### EDA on Airport Data
- ident: identity of airport
- type: the category of the airport
- airport_name: the name of the airport
- elevation_ft: the elevation ft
- continent: geograpgic metada - continent
- iso_country: geograpgic metada - iso country code
- municipality: geograpgic metada - city
- gps_code: geograpgic metada - gps code
- iata_code: geograpgic metada - iata code
- local_code: geograpgic metada - local code
- region: geograpgic metada - regoin (states in the US)
- municipality_region: combined field, which is consist of municipality and region

In [1]:
import os
import datetime
import pandas as pd
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 1028)
import configparser

import boto3

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, DoubleType, StringType, IntegerType, DateType, LongType, MapType

In [2]:
# config = configparser.ConfigParser()
# config.read('/home/workspace/capstone.cfg')

['/home/workspace/capstone.cfg']

In [3]:
# os.environ["AWS_ACCESS_KEY_ID"] = config['AWS']['AWS_ACCESS_KEY_ID']
# os.environ["AWS_SECRET_ACCESS_KEY"] = config['AWS']['AWS_SECRET_ACCESS_KEY']
# input_data_dir = config['S3']['INPUT_S3_BUCKET']

In [4]:
# Init a SparkSession object
spark = SparkSession \
    .builder \
    .config('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:2.7.0') \
    .getOrCreate()

In [5]:
airport_code =spark.read \
    .format('csv') \
    .options(header='true', inferSchema='true', encoding="ISO-8859-1") \
    .load(f'{input_data_dir}/airport-codes_csv.csv')

In [6]:
airport_code = airport_code.withColumnRenamed('name','airport_name')

In [7]:
# airport_code.printSchema()

root
 |-- ident: string (nullable = true)
 |-- type: string (nullable = true)
 |-- airport_name: string (nullable = true)
 |-- elevation_ft: integer (nullable = true)
 |-- continent: string (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- iso_region: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- local_code: string (nullable = true)
 |-- coordinates: string (nullable = true)



In [8]:
# airport_code.limit(10).toPandas()

,ident,type,airport_name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11,NA,US,US-PA,Bensalem,00A,None,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435,NA,US,US-KS,Leoti,00AA,None,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450,NA,US,US-AK,Anchor Point,00AK,None,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820,NA,US,US-AL,Harvest,00AL,None,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237,NA,US,US-AR,Newport,None,None,None,"-91.254898, 35.6087"
5,00AS,small_airport,Fulton Airport,1100,NA,US,US-OK,Alex,00AS,None,00AS,"-97.8180194, 34.9428028"
6,00AZ,small_airport,Cordes Airport,3810,NA,US,US-AZ,Cordes,00AZ,None,00AZ,"-112.16500091552734, 34.305599212646484"
7,00CA,small_airport,Goldstone /Gts/ Airport,3038,NA,US,US-CA,Barstow,00CA,None,00CA,"-116.888000488, 35.350498199499995"
8,00CL,small_airport,Williams Ag Airport,87,NA,US,US-CA,Biggs,00CL,None,00CL,"-121.763427, 39.427188"
9,00CN,heliport,Kitchen Creek Helibase Heliport,3350,NA,US,US-CA,Pine Valley,00CN,None,00CN,"-116.4597417, 32.7273736"


In [9]:
split_coordinates = F.split(airport_code['coordinates'], ', ')
airport_code = airport_code \
    .withColumn('Latitude', split_coordinates.getItem(0)) \
    .withColumn('Longitude', split_coordinates.getItem(1)) \
    .drop('coordinates')

In [10]:
# airport_code.limit(10).toPandas()

,ident,type,airport_name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,Latitude,Longitude
0,00A,heliport,Total Rf Heliport,11,NA,US,US-PA,Bensalem,00A,None,00A,-74.93360137939453,40.07080078125
1,00AA,small_airport,Aero B Ranch Airport,3435,NA,US,US-KS,Leoti,00AA,None,00AA,-101.473911,38.704022
2,00AK,small_airport,Lowell Field,450,NA,US,US-AK,Anchor Point,00AK,None,00AK,-151.695999146,59.94919968
3,00AL,small_airport,Epps Airpark,820,NA,US,US-AL,Harvest,00AL,None,00AL,-86.77030181884766,34.86479949951172
4,00AR,closed,Newport Hospital & Clinic Heliport,237,NA,US,US-AR,Newport,None,None,None,-91.254898,35.6087
5,00AS,small_airport,Fulton Airport,1100,NA,US,US-OK,Alex,00AS,None,00AS,-97.8180194,34.9428028
6,00AZ,small_airport,Cordes Airport,3810,NA,US,US-AZ,Cordes,00AZ,None,00AZ,-112.16500091552734,34.305599212646484
7,00CA,small_airport,Goldstone /Gts/ Airport,3038,NA,US,US-CA,Barstow,00CA,None,00CA,-116.888000488,35.350498199499995
8,00CL,small_airport,Williams Ag Airport,87,NA,US,US-CA,Biggs,00CL,None,00CL,-121.763427,39.427188
9,00CN,heliport,Kitchen Creek Helibase Heliport,3350,NA,US,US-CA,Pine Valley,00CN,None,00CN,-116.4597417,32.7273736


In [11]:
country_iso_code =spark.read \
    .format('csv') \
    .options(header='true', inferSchema='true', encoding="ISO-8859-1") \
    .load(f'{input_data_dir}/iso_2digit_alpha_country_codes.csv')

In [12]:
# country_iso_code.printSchema()

root
 |-- Code: string (nullable = true)
 |-- Name: string (nullable = true)



In [13]:
new_airport_code = airport_code \
    .join(country_iso_code, airport_code.iso_country==country_iso_code.Code, 'left') \
    .drop('Code') \
    .withColumnRenamed('Name', 'iso_country_fullname')

In [14]:
# new_airport_code.limit(10).toPandas()

,ident,type,airport_name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,Latitude,Longitude,iso_country_fullname
0,00A,heliport,Total Rf Heliport,11,NA,US,US-PA,Bensalem,00A,None,00A,-74.93360137939453,40.07080078125,United States
1,00AA,small_airport,Aero B Ranch Airport,3435,NA,US,US-KS,Leoti,00AA,None,00AA,-101.473911,38.704022,United States
2,00AK,small_airport,Lowell Field,450,NA,US,US-AK,Anchor Point,00AK,None,00AK,-151.695999146,59.94919968,United States
3,00AL,small_airport,Epps Airpark,820,NA,US,US-AL,Harvest,00AL,None,00AL,-86.77030181884766,34.86479949951172,United States
4,00AR,closed,Newport Hospital & Clinic Heliport,237,NA,US,US-AR,Newport,None,None,None,-91.254898,35.6087,United States
5,00AS,small_airport,Fulton Airport,1100,NA,US,US-OK,Alex,00AS,None,00AS,-97.8180194,34.9428028,United States
6,00AZ,small_airport,Cordes Airport,3810,NA,US,US-AZ,Cordes,00AZ,None,00AZ,-112.16500091552734,34.305599212646484,United States
7,00CA,small_airport,Goldstone /Gts/ Airport,3038,NA,US,US-CA,Barstow,00CA,None,00CA,-116.888000488,35.350498199499995,United States
8,00CL,small_airport,Williams Ag Airport,87,NA,US,US-CA,Biggs,00CL,None,00CL,-121.763427,39.427188,United States
9,00CN,heliport,Kitchen Creek Helibase Heliport,3350,NA,US,US-CA,Pine Valley,00CN,None,00CN,-116.4597417,32.7273736,United States


In [15]:
# airport_code.filter(F.col('municipality')=='New York').toPandas()

,ident,type,airport_name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,Latitude,Longitude
0,6N5,heliport,East 34th Street Heliport,10.0,NA,US,US-NY,New York,6N5,TSS,6N5,-73.97209930419922,40.74259948730469
1,6N6,seaplane_base,Evers Seaplane Base,NaN,NA,US,US-NY,New York,6N6,None,6N6,-73.81620025634766,40.84590148925781
2,6N7,seaplane_base,New York Skyports Inc Seaplane Base,NaN,NA,US,US-NY,New York,None,QNY,6N7,-73.9729,40.734001
3,JPB,closed,Pan Am Building Heliport,870.0,NA,US,US-NY,New York,None,JPB,None,-73.9765,40.7533
4,JRA,heliport,West 30th St. Heliport,7.0,NA,US,US-NY,New York,KJRA,JRA,JRA,-74.007103,40.754501
5,JRB,heliport,Downtown-Manhattan/Wall St Heliport,7.0,NA,US,US-NY,New York,KJRB,JRB,JRB,-74.00900269,40.70119858
6,KJFK,large_airport,John F Kennedy International Airport,13.0,NA,US,US-NY,New York,KJFK,JFK,JFK,-73.77890015,40.63980103
7,KLGA,large_airport,La Guardia Airport,21.0,NA,US,US-NY,New York,KLGA,LGA,LGA,-73.87259674,40.77719879
8,KNOP,closed,Floyd Bennett Field,16.0,NA,US,US-NY,New York,None,None,None,-73.890999,40.591
9,NK39,heliport,One Police Plaza Heliport,244.0,NA,US,US-NY,New York,NK39,None,NK39,-73.99960327148438,40.71260070800781


In [16]:
spark.stop()